<a href="https://colab.research.google.com/github/Visionlogic-AI/CV_Model_Deployment/blob/master/Robust_Computer_Vision_Model_Deployment%5BTemplate%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we are goingt to go over the proper steps in order to deploy robust computer vision applications for a production environment.

SAVE MODEL FOR TENSORFLOW SERVING

In [0]:
#saving model
model= create_maskrcnn_architecture_model(input_shape= INPUTE_SHAPE)
model.load_weights('./model_weights/mask_rcnn_model_wwt.h5')
export_path= './tf_saved_models'

tf.saved_model.simple.save(
keras.backend.get_sessions(),
export_path,
inputs={'input_image': model.inpout},
outputs= {t.name:t for t in model.outputs})

#view saved model(s)...if more than one model is being saved
! tree --du -h .

Use tensorflow command line interface (CLI) tool to quikcly inspect the input and output specifications of our model

In [0]:
!export PYTHONWARNINGS='ignore' && save _model_cli show --dir {'./tf_saved_model/'} --all

SERVE MODEL(S) WITH GPU INFERENCE
*We will leverage Docker to setup our tensorflow serving system. 

Pulling TF Serivng GPU Image:
First, we have to esure that docker is installed in our system (or in the coud).
Once the proper docker file has been installed into our system, we can use the following code to pull the latest version of TF Serving on GPU's.

In [0]:
!docker pull tensorflow/serving:latest-gpu

Then we can check to ensure the image is present in our system using the following command:

In [0]:
!docker images

We are now ready to start serving models with TF Serving. We can do this by running the docker image that we have just downloaded.
*This can be done directly in Jupyter Notebooks using the following code below. (In practice it is better to run it from the command terminal)

In [0]:
%%bash --bg
docker run --runtime=nvidia -p 8501:8501 \
    --mount type= bind,source=/home/jupyter/tensorflow_serving/tf_saved_models,target=//home/jupyter/tensorflow_serving/tf_saved_models\
    --mount type=bind,source=/home/jupyter/tensorflow_serving/models.conf,target=/home/jupyter/tensorflow_serving/models/conf\
    -t tensorflow/serving:latest-gpu --model_config_file=/home/jupyter/tensorflow_servning/models.conf

#Then we can use the command "in Docker" to ensure the container is up and running

In [0]:
!docker ps -all # use this in docker!

Finally, we can chec the logs inside of Docker to verify that everything is working perfectly

In [0]:
!docker logs 7d4b091ccefa | tail -n 15

#this verifies the fact that TF Serving will be using the GPU on our system for inference!

MODEL WARMUP
We can leverage our previously implemented code to warmup our model(s).

In [0]:
warmup_data= np.load('serve_warmup_data.npy')
warmup_model_serve(warmup_data)

BECNHMARK MODEL SERVING REQUESTS:
Here, we will take all of our test images and send a single request to check model serving time for inference using our GPU.

In [0]:
%time

sample_test_data= test_images
sample_test_labels= test_labels
IMG_DIMS= (32,32) 
sample_test_data_processed = (np.array([resize_image_array(img,
                                                          img_size_dims= IMG_DIMS)
                                       for img in np.stack([sample_test_data]*3,
                                                          axis= -1)])) / 255.
data = json.dumps({'signature_name': 'serving_default',
                  'instances': sample_test_data_processed.tolist()})

HEADERS= {'content-type': 'application/json'}
MODEL_API_URL= 'http:localhost:8501/v1/models/maskrcnn_model_serving/versions/:predict'

json_response= requests.post(MODEL_API_URL, data=data, headers= HEADERS)
predictions= json.loads(json_response.text)['predictions']
predictions= np.argmax(np.array(predictions), axis=1)
predictions_labels = [class_names[p] for p in predictions]
len(prediction_labels)

Lets now perform REAL BECNHMARK TESTING
Now lets, consider x amount of "seperate" requests for inference of a single image to be detected each time.
(This will show us how much time it would tae for tf serving to serve these x amount of requests)

In [0]:
#create serving function
def predict_maskrcnn_model_serving(img, img_dims=(32,32), label_map=class_names):
    #create sample image to prcoess
    sample_img_processed= (np.array([resize_image_array(img,
                                                       img_size_dims=img_dims)
                                    for img in np.stack([[img]]*3,
                                                       axis= -1)]))/ 255.
    
    data= json.dumps({'signature_name': 'serving_default',
                     'instances': sample_img_processed.tolist()})
    
    HEADERS= {'content-type': 'application/json'}
    
    MODEL_API_URL= 'http://localhost:8501/v1/model/maskrcnn_model_serving/:predict'
    json_response= requests.post(MODEL_API_URL, data=data, headers= HEADERS)
    prediction= json.loads(json_response_text)['predictions']
    prediction= np.argmax(np.array(prediction), axis=1)[0]
    return label_map[prediction]

#bench for x amount of request
%%time

pred_labels= []
for img in tqdm(test_images):
    pred_label= predict_maskrcnn_model_serving(img)
    pred_labels.append(img)
len(pred_labels)

Now lets try an interesting comparison in order to see how long it takes using the regular api from Keras.

In [0]:
#create keras serving function
INPUT_SHAPE_RN= (32,32,3)
model= create_maskrcnn_architecture_model(input_shape=INPUT_SHAPE_RN)
model.load_weight('./model_weights/maskrcnn_model_wt.h5')  #make sure this is the name of your exact model

def predict_maskrcnn_model_regular(img, img_dims=(32,32), label_map= class_names):
    #create sample image processing
    sample_img_prcoessed= (np.array([resize_image_array(img,
                                                       img_size_dims= img_dims)
                                    for img in np.stack([[img]]*3,
                                                       axis= -1)]))/ 255.
    prediction= model.predict(sample_img_processed)
    prediction = np.argmax(np.array(prediction), axis=1)[0]
    return label_map[prediction]

#benchmrk x number of request
%%time

pred_labels= []
for img in tqdm(test_images):
    pred_lael= predict_maskrcnn_model_regular(img)
    pred_labels.append(img)
len(pred_labels)

LEVERAGING FLASK TO BUILD A WEB SERVICE/API ON TOP OF TF SERVING 
This will allow:
    - allow us to accept real world images 
    - conduct the proper pre-precesing
    - call TF Serving
    - post process rhe response
    - and then send the final json response to the end user
   
Do Note: We can even dockerize and deploy the Flask ApI on Kubernetes or use a WSGI server like Gunicorn to scale and improve performance.

In [0]:
#create API with Flask
#load dependencies
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import requests
import base64
import json
from io import BytesIO
from flask import Flask, requests, jsonify
from flask_cors import CORS
import tensorflow as tf
from tensorflow import Keras
import cv2

#TF Serving Assets
HEADERS= {'content-type': 'application/json'}
MODEL_API_URL= 'http:localhost:8501/v1/models/maskrcnn_model_serving/:predict'
CLASS_NAMES= ['list_class_names_here']  #here is where we list all of the class names for our model


#Instantiate the Flask App
app= Flask(__name__)
CORS(app)

#Image resizing utils
def resize_image_array(img, img_size_dims):
    img= cv2.resize(img, dsize=img_size_dims,
                   interpolation= cv2.INTER_CUBIC)
    IMG= np.array(img, dtype=np.float32)
    return img

#model warmup function
def warmup_model_serve(warmup_data, img_dims=(32,32)):
    warmup_data_processed= (np.array([resize_image_array(img,
                                                       img_size_dims= img_dims)
                                    for img in np.stack([warmup_data]*3,
                                                       axis= -1)])) / 255.
    
    data= json.dumps({'signature_name': 'serving_default',
                     'instances': warmup_data_processed.tolist()})
    
    json_response= requests.post(MODEL_API_URL, data= data, headers= HEADERS)
    predictions= json.loads(json_response.text)['predictions']
    print('Model warmup complete ') #log this in actual production
    
#TF lazy loads so we warmup model with sample data
#this runs as soon as we setup our web service to run
warmup_data= np.load('serve_warmup_data.npy')
warmup_model_serve(warmup_data)

#Livesness Test 
@app.route('/maskrcnn_detection/api/v1/liveness', methods=['GET', 'POST'])  #make sure this mathes your model name
def liveness():
    return 'API Live!'

#Model inference endpoint
@app.route('maskrcnn_detection/api/v1/model_predict', methods= ['POST'])
def maskrcnn_detection_model():
    img= np.array([keras.preprocessing.image.img_to_array(
    keras.preprocessing.image.load_img(BytesIO(base64.b64decode(request.form['b64_img'])),
                                      target_size=(32,32))) / 255.])
    data= json.dumps({'signature_name': 'serving_default',
                     'instances': img.tolist()})
    json_response= requests.post(MODEL_API_URL, data=data, headers= HEADERS)
    prediction= json.loads(json_response.text)['predictions']
    prediction= np.argmax(np.asarray(predictions), axis=1)[0]
    prediction= CLASS_NAMES[prediction]
    
    return jsonify({'vision_prediction': prediction})  #this name can be anything you choose

#running REST interface, port=5000 for direct test
#use debug= True when debugging
if __name__== '__main__':
    app.run(debug= False, host= '0.0.0.0', port=5000)
    

We store this file as app.py in our server which forms the base of our api

STORE DOCKER CONTAINER FOR TF SERVING:
Check and restart the docker container for TF Serving if not's already up and running

In [0]:
#run this code in docker command
!docker start 7d4b091ccefa
!docker ps -all

START OUR COMPUTER VISION OBJECT DETECTION/RECOGNITION MODEL
*Now we need to start our web service. 
In production, it is vital that we do not use the default web server provided by flask but a better production ready WSGI server, like Gunicorn.
We start our web service using the following command "from terminal".

In [0]:
/home/jupyter/tensorflow_serving$ gunicorn -b 0.0.0.0:5000 app:app -w 8

We leverage multiple workers to serve more requests as needed. 
Lets now check if our API is live using the "liveness test endpoint".

In [0]:
r= requests.get('http://0.0.0.0:5000/vision_prediction/api/v1/liveness')
r.status_code, r.text

In [0]:
#SERVE SAMPLE OBJECT DETECTION/RECOGNITION WITH WEB SERVICE
plt.imshow(cv2.cvtColor(cv2.imread('image.jpg'),
                       cv2.COLOR_BGR2RRGB))

Once the sample image is what we are looking for, we can then leverage our API to serve the model prediction.

In [0]:
import base64
import requests

with open('image.jpg', 'rb') as imageFile:
    img_b64enc= base64.b64encoder(imageFile.read())
    data= {'b64_img': img_b64enc}

API_URL= 'http://0.0.0.0:5000/vision_prediction/api/v1/model_predict'  #make sure your model matches from what you created
#sending post request and saving response as response object
r = requests.post(url= API_URL, data=data)
r.json()

Once the  prediction we are looking for is accurate, we then need to benchmark our web service
Lets check out how much time it takes to process 10000 requests now.

In [0]:
%%time

responses= []
for i in tqdm(range(10000)):
    API_URL= 'http://0.0.0.0:5000/vision_prediction/api/v1/model_predict'  #check and solidify your exact name
    #sending post request and saving response as response object
    r= request.append(r.json())
    
len(responses)
print('Inference time per image: {} ms'.format((326/10000) *1000))TYUIOP[] \